In [47]:
import gzip
import os

import pandas as pd
import numpy as np
import dask.dataframe as dd
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

In [3]:
# def decompress_gz_files(input_directory, output_directory):
#     # Vérifier si le répertoire de destination existe, sinon le créer
#     if not os.path.exists(output_directory):
#         os.makedirs(output_directory)

#     # Parcourir les fichiers dans le répertoire source
#     for filename in os.listdir(input_directory):
#         if filename.endswith(".gz"):
#             gz_file_path = os.path.join(input_directory, filename)
#             decompressed_file_path = os.path.join(output_directory, filename[:-3])  # Retirer ".gz"

#             # Ouvrir le fichier .gz et décompresser son contenu dans le répertoire de destination
#             with gzip.open(gz_file_path, 'rb') as gz_file:
#                 with open(decompressed_file_path, 'wb') as decompressed_file:
#                     decompressed_file.write(gz_file.read())
#             print(f"Décompressé : {filename} dans {output_directory}")

# # Appel de la fonction
# decompress_gz_files("../gzip", "../Data")

In [39]:
spark = SparkSession.builder \
        .appName("my_job") \
        .master("local[*]") \
        .getOrCreate()

In [40]:
spark

In [45]:
data_name_sp = spark.read.option("header", "true").csv("../Data/name.basics.tsv", sep="\t", inferSchema=True)
data_name_sp.show(5)

+---------+---------------+---------+---------+--------------------+--------------------+
|   nconst|    primaryName|birthYear|deathYear|   primaryProfession|      knownForTitles|
+---------+---------------+---------+---------+--------------------+--------------------+
|nm0000001|   Fred Astaire|     1899|     1987|actor,miscellaneo...|tt0072308,tt00504...|
|nm0000002|  Lauren Bacall|     1924|     2014|actress,soundtrac...|tt0037382,tt00752...|
|nm0000003|Brigitte Bardot|     1934|       \N|actress,music_dep...|tt0057345,tt00491...|
|nm0000004|   John Belushi|     1949|     1982|actor,writer,musi...|tt0072562,tt00779...|
|nm0000005| Ingmar Bergman|     1918|     2007|writer,director,a...|tt0050986,tt00839...|
+---------+---------------+---------+---------+--------------------+--------------------+
only showing top 5 rows



In [49]:
data_name_sp = data_name_sp.select(
    [when(col(c) == '\\N', None).otherwise(col(c)).alias(c) for c in data_name_sp.columns]
)

In [43]:
data_name_sp.count()

13840848

In [50]:
data_name_sp.select([sum(col(c).isNull().cast("int")).alias(c) for c in data_name_sp.columns]).show()

+------+-----------+---------+---------+-----------------+--------------+
|nconst|primaryName|birthYear|deathYear|primaryProfession|knownForTitles|
+------+-----------+---------+---------+-----------------+--------------+
|     0|         50| 13213908| 13606193|          2677679|       1554685|
+------+-----------+---------+---------+-----------------+--------------+

